# Qiskit config

In [1]:
from qiskit import IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile

backend = Aer.get_backend('qasm_simulator')

shots   = 8192
backend_name = backend.name()
backend_config = backend.configuration()
backend_qubits = backend_config.n_qubits

# Experiment

In [3]:
import numpy as np
import sys
sys.path.append('../../')
from qclib.state_preparation.schmidt import initialize
from qclib.state_preparation.schmidt import initialize_original

result = {}
for n in range(4, 11):
    N = 2**n
    print('\nn =', n)
        
    input_state = ((1.0 - 0.01) * np.random.rand(N) + 0.01) + ((1.0 - 0.01) * np.random.rand(N) + 0.01) * 1j
    input_state = input_state/np.linalg.norm(input_state)

    result[n] = []
    
    r = 1
    circuit1 = initialize(input_state, rank = r)
    circuit2 = initialize_original(input_state)
    
    transpiled_circuit1 = transpile(circuit1, basis_gates=['u1','u2','u3', 'cx'], optimization_level=3)
    transpiled_circuit2 = transpile(circuit2, basis_gates=['u1','u2','u3', 'cx'], optimization_level=3)
    n_cx1 = transpiled_circuit1.count_ops()['cx']
    n_dp1 = transpiled_circuit1.depth()
    n_cx2 = transpiled_circuit2.count_ops()['cx']
    n_dp2 = transpiled_circuit2.depth()

    result[n].append(['isometria', n_cx1, n_dp1])
    result[n].append(['normal', n_cx2, n_dp2])

    print('\t {0}\t CNOTs = {1}\t depth = {2}'.format('isometria', n_cx1, n_dp1))
    print('\t {0}\t\t CNOTs = {1}\t depth = {2}'.format('normal', n_cx2, n_dp2))
        


n = 4
	 isometria	 CNOTs = 3	 depth = 4
	 normal		 CNOTs = 9	 depth = 11

n = 5
	 isometria	 CNOTs = 6	 depth = 9
	 normal		 CNOTs = 30	 depth = 53

n = 6
	 isometria	 CNOTs = 9	 depth = 10
	 normal		 CNOTs = 54	 depth = 57

n = 7
	 isometria	 CNOTs = 14	 depth = 11
	 normal		 CNOTs = 150	 depth = 243

n = 8
	 isometria	 CNOTs = 24	 depth = 15
	 normal		 CNOTs = 253	 depth = 247

n = 9
	 isometria	 CNOTs = 108	 depth = 165
	 normal		 CNOTs = 661	 depth = 1039

n = 10
	 isometria	 CNOTs = 203	 depth = 189
	 normal		 CNOTs = 1089	 depth = 1078


# Plot